In [1]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import re
# await page.screenshot(path="example.png")
# await browser.close()
# await pw.stop()

url1='https://m.facebook.com/wasitl'
url2='https://m.facebook.com/sothna.vicharueng'
url3='https://m.facebook.com/zuck'
url=url3

In [2]:
import time
async def get_html(url):
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless = False)
        page = await browser.new_page()
        html = await page.goto(url)
        await page.evaluate(
            '''
            var intervalID = setInterval(function () {
            var scrollingElement = (document.scrollingElement || document.body);
            scrollingElement.scrollTop = scrollingElement.scrollHeight;
            }, 200);
            ''')
        prev_height = None
        """ 
        sometime facebook show page to login 
        i make click to remove login page and continue scroll
        
        make the mouse scroll up to make program restart
        """
        try:
            item_close = await page.locator("[aria-label=\"Close\"]").click()
            if item_close is not None:
                item_close.click()
                page.mouse.wheel(-1000, 0)
        except :
            pass
        while True:
            curr_height = await page.evaluate('(window.innerHeight + window.scrollY)')
            if not prev_height:
                prev_height = curr_height
                # time.sleep(3)
            elif prev_height == curr_height:
                await page.evaluate('clearInterval(intervalID)')
                break
            else:
                prev_height = curr_height
                # time.sleep(10)
        
        selector = await page.query_selector('div#root')
        html = await selector.inner_html()
    return html

In [3]:
def get_data(html):
    soup = BeautifulSoup(html, "html.parser")
    username = soup.select_one('div#cover-name-root').text

    images = soup.select_one('div#profile_photos_unit').find_all('img')
    img_urls = [img['src'] for img in images]

    main = soup.select_one('div#main_column')
    
    data={}
    for i in main.select('div[data-sigil=profile-card]'):
        role=i.select_one('div[role=heading]').text
        # print(f'Role:{role}')
        
        ## exprerience
        exp=[]
        for j in i.select('div._5cds._2lcw'):
            d={}
            for k in j.select('div span'):    
                key='.'.join(k.get('class',''))
                if re.search('\w{1,}', key):
                    # print(k)
                    d[key]=k.text
            exp.append(d)
        if len(exp):
            data[role]=exp
        
        ## place lived
        live=[]
        for j in i.select('div._2swz._2lcw'):
            d={}
            for k in j.select('div header h4'):
                key='.'.join(k.get('class','h4'))
                if re.search('\w{1,}', key):
                    # print(k)
                    d[key]=k.text
            live.append(d)
        if len(live):
            data[role]=live
    return {'name':username, 'card':data, 'images':img_urls, }
    

In [4]:
html= await get_html(url1)
get_data(html)

{'name': 'Wasit Limprasert',
 'card': {'Work': [{'_52jd._52jb._52jh._3-8_': 'Thammasat University',
    '_52jc._52ja': 'Lecturer',
    '_52jc._52j9': 'May 2014 - Present',
    '_52jc._52jb': 'Computer Science'},
   {'_52jd._52jb._52jh._3-8_': 'NECTEC NSTDA',
    '_52jc._52ja': 'Researcher',
    '_52jc._52j9': 'Rangsit, Pathum Thani, Thailand',
    '_52jc._52jb': 'Currently working on Computer Graphics and Visualization'},
   {'_52jd._52jb._52jh._3-8_': 'Asian Institute of Technology',
    '_52jc._52ja': 'computer lab supervisor',
    '_52jc._52j9': '2007 - 2008'},
   {'_52jd._52jb._52jh._3-8_': 'คณะวิทยาศาสตร์และเทคโนโลยี มหาวิทยาลัยธรรมศาสตร์ ศูนย์รังสิต'}],
  'Education': [{'_52jd._52jb._52jh._3-8_': 'Heriot-Watt University',
    '_52jc._52ja': 'vision and signal processing',
    'concs.mfss.fcg': 'vision and signal processing',
    '_52jc._52j9': 'School year 2012'},
   {'_52jd._52jb._52jh._3-8_': 'Asian Institute of Technology',
    '_52jc._52ja': 'Microelectronics',
    'concs.mfs

In [5]:
html= await get_html(url2)
get_data(html)

{'name': 'Sothana Vicharueng (Nai)',
 'card': {'Education': [{'_52jd._52jb._52jh._3-8_': 'วินิตศึกษาในพระราชูปถัมภ์',
    '_52jc._52ja': 'High School'},
   {'_52jd._52jb._52jh._3-8_': 'โรงเรียนสวนกุหลาบวิทยาลัย รังสิต',
    '_52jc._52ja': 'High School'}],
  'Places lived': [{'h.4': 'Lopburi',
    '_52jc._52ja._52jg': 'Current town/city'},
   {'h.4': 'Bangkok, Thailand', '_52jc._52ja._52jg': 'Home town'}]},
 'images': ['https://scontent.fbkk5-6.fna.fbcdn.net/v/t1.6435-9/96748792_1325680447623063_218624890612219904_n.jpg?stp=cp0_dst-jpg_e15_p160x160_q65&_nc_cat=101&ccb=1-7&_nc_sid=efc22e&_nc_ohc=b5Vj_ed-YKYAX9ahTOT&_nc_ht=scontent.fbkk5-6.fna&oh=00_AfCNbEGMX8mXtfSe93k5dvfPYlBOwTn-ZlMJLkfTBhobVA&oe=656B923D',
  'https://scontent.fbkk5-3.fna.fbcdn.net/v/t1.18169-9/20840881_674954342695680_1598823011790338011_n.jpg?stp=cp0_dst-jpg_e15_p160x160_q65&_nc_cat=105&ccb=1-7&_nc_sid=efc22e&_nc_ohc=DN2vMEgyHBEAX82Y-vH&_nc_ht=scontent.fbkk5-3.fna&oh=00_AfCR5OIVtl3GtOEFQaSR4hCr3Cj5ImJ7AfYryEGaYSOEvg&o

In [6]:
html= await get_html(url3)
get_data(html)

{'name': 'Mark Zuckerberg',
 'card': {'Work': [{'_52jd._52jb._52jh._3-8_': 'Chan Zuckerberg Initiative',
    '_52jc._52j9': '1 December 2015 - Present'},
   {'_52jd._52jb._52jh._3-8_': 'Meta',
    '_52jc._52ja': 'Founder and CEO',
    '_52jc._52j9': 'Palo Alto, California',
    '_52jc._52jb': 'Bringing the world closer together.'}],
  'Education': [{'_52jd._52jb._52jh._3-8_': 'Harvard University',
    '_52jc._52ja': 'Computer Science and Psychology',
    'concs.mfss.fcg': 'Computer Science and Psychology',
    '_52jc._52j9': '30 August 2002 - 30 April 2004'},
   {'_52jd._52jb._52jh._3-8_': 'Phillips Exeter Academy',
    '_52jc._52ja': 'Classics',
    'concs.mfss.fcg': 'Classics',
    '_52jc._52j9': 'School year 2002'},
   {'_52jd._52jb._52jh._3-8_': 'Ardsley High School',
    '_52jc._52ja': 'High School',
    '_52jc._52j9': 'September 1998 - June 2000'}],
  'Places lived': [{'h.4': 'Palo Alto, California',
    '_52jc._52ja._52jg': 'Current town/city'},
   {'h.4': 'Dobbs Ferry, New York

In [7]:
#https://github.com/sqlalchemy/sqlalchemy/discussions/7634#discussioncomment-2073508